# Silnik

- WOS 175/L SILNIK WYSOKOPRĘŻNY QSB6,7-C240 CUMMINS praca od 21.05.2019 do 11.02.2021,
- LK3 045/L SILNIK TCD2013L062V praca od 21.05.2019 do 04.03.2021,
- LK3 050/L SILNIK TCD2013L062V praca od 11.09.2019 do 19.05.2020,

In [1]:
import pandas as pd

import datetime
import pyarrow

## SILNIK 045/L

In [2]:
df = pd.read_parquet('../data/SILNIK_045L_2021_01.parquet', engine='pyarrow')
df.head(3)

,maszyna,data,czas,engoilp,enghours,fuelus,intakep,intaket,trnlup,groilp,...,breakp,hyddrv,trnaut,hydoilt,hydoilp,tempin,engrpm,engcoolt,engtps,trnbps
0,KLDSMG_WOS___175L,2021/01/01,00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KLDSMG_WOS___175L,2021/01/01,01:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLDSMG_WOS___175L,2021/01/01,01:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_wos174 = pd.DataFrame(columns=df.columns)

for file_date in ['2021_03','2020_11','2020_12','2021_01','2021_02']:
    df_tmp = pd.read_parquet('../data/SILNIK_045L_' + file_date + '.parquet', engine='pyarrow')
    df_wos174 = pd.concat([df_wos174, df_tmp], axis=0)

In [4]:
df_wos174_copy = df_wos174.copy()

In [5]:
df_wos174 = df_wos174_copy.copy()

#zmienna wróżka; bo jest narastająca, zatem jak blisko konkretnej daty się psują to jest tu zakodowane info o statusie
df_wos174 = df_wos174.drop(columns=['enghours'])

#mało wartości
df_wos174 = df_wos174.drop(columns=['trnaut'])

#cleaning if all values are NaN
#https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan
df_wos174 = df_wos174.dropna(thresh=len(df_wos174.columns)-3) 

#datatime column
df_wos174['dt'] = pd.to_datetime(df_wos174['data'] + ' ' + df_wos174['czas'])
df['data'] = df['data'].astype('datetime64[ns]')

#sorting descending by date
df_wos174 = df_wos174.sort_values(by=['dt'], ascending = False).reset_index(drop=True)

#delete observations after failure
df_wos174 = df_wos174[df_wos174['dt']<'2022-03-05']

#prepare aggregats
df_wos174_1min = df_wos174.resample('1min', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')
df_wos174_1hour = df_wos174.resample('1H', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')
df_wos174_1day = df_wos174.resample('1D', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')

In [6]:
# tworzymy cechy 6h wcześniej, 12h wcześniej 24h wcześniej 48h wcześniej, 7dni wcześniej
# i agregaty mean (jak pyknie dodamy min i max)

df_tmp_6h = df_wos174_1hour.rolling(6).mean()
df_tmp_6h.columns = ['avg6_'+ col for col in list(df_tmp_6h.columns)]

df_tmp_12h = df_wos174_1hour.rolling(12).mean()
df_tmp_12h.columns = ['avg12_'+ col for col in list(df_tmp_12h.columns)]

df_tmp_24h = df_wos174_1hour.rolling(24).mean()
df_tmp_24h.columns = ['avg24_'+ col for col in list(df_tmp_24h.columns)]

df_tmp_48h = df_wos174_1hour.rolling(48).mean()
df_tmp_48h.columns = ['avg48_'+ col for col in list(df_tmp_48h.columns)]

df_tmp_168h = df_wos174_1hour.rolling(168).mean()
df_tmp_168h.columns = ['avg168_'+ col for col in list(df_tmp_168h.columns)]

In [7]:
df_wos174_1hour_hist = pd.concat([df_wos174_1hour, df_tmp_6h, df_tmp_12h, df_tmp_24h, df_tmp_48h, df_tmp_168h],axis=1)

In [8]:
df_wos174_1hour_hist = df_wos174_1hour_hist.reset_index()
# status good / bad. If 2 weeks before failture then bad else good

df_wos174_1hour_hist['status'] = df_wos174_1hour_hist['dt'].apply(lambda x: 0 if x < datetime.datetime(2021,3,5)-datetime.timedelta(days=14) else 1)

# buffor - 2 weeks between good and bads
df_wos174_1hour_hist['buffor'] = df_wos174_1hour_hist['dt'].apply(lambda x: 1 if x < datetime.datetime(2021,3,5)-datetime.timedelta(days=30) else 0)

# not longer then 3 months before failure
df_wos174_1hour_hist['notLonger'] = df_wos174_1hour_hist['dt'].apply(lambda x: 0 if x < datetime.datetime(2021,3,5)-datetime.timedelta(days=90) else 1)


In [ ]:
df_wos174_1hour_hist_final = pd.concat([df_wos174_1hour_hist[df_wos174_1hour_hist['status']==1],
                                        df_wos174_1hour_hist[(df_wos174_1hour_hist['status']==0) & (df_wos174_1hour_hist['buffor']==1) & (df_wos174_1hour_hist['notLonger']==1)]
                                       ], axis=0)

print(f'shape: {df_wos174_1hour_hist_final.shape}')

df_wos174_1hour_hist_final.status.value_counts()

In [ ]:
#df_wos174_1hour_hist_final = df_wos174_1hour_hist_final.drop(columns=['index']).iloc[:,1:-2].reset_index(drop=True)
df_wos174_1hour_hist_final = df_wos174_1hour_hist_final.iloc[:,1:-2].reset_index(drop=True)
df_wos174_1hour_hist_final.head(3)

In [ ]:
df_wos174_1hour_hist_final.to_csv('../data/df_silnik175.csv', sep="|", index=False)